In [109]:
import pandas as pd
import numpy as np
from datetime import date

In [12]:
df = pd.read_csv('D:/Data Science/FIFA21-Data-Analysis/fifa21_raw_data.csv', low_memory = False)

In [38]:
#print(df.head())
#print(df.columns)
#print(df.shape)
#print(df.isnull().sum())
#print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18979 entries, 0 to 18978
Data columns (total 79 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   photoUrl          18979 non-null  object 
 1   LongName          18979 non-null  object 
 2   playerUrl         18979 non-null  object 
 3   Nationality       18979 non-null  object 
 4   Positions         18979 non-null  object 
 5   Name              18979 non-null  object 
 6   Age               18979 non-null  int64  
 7   ↓OVA              18979 non-null  int64  
 8   POT               18979 non-null  int64  
 9   Team & Contract   18979 non-null  object 
 10  ID                18979 non-null  int64  
 11  Height            18979 non-null  object 
 12  Weight            18979 non-null  object 
 13  foot              18979 non-null  object 
 14  BOV               18979 non-null  int64  
 15  BP                18979 non-null  object 
 16  Growth            18979 non-null  int64 

* Convert weight from lbs to kg and create new column

In [14]:
df['Weight (in kg)'] = df.Weight.str.replace('lbs', '')
df['Weight (in kg)'] = df['Weight (in kg)'].astype(float) * 0.453592
print(df['Weight (in kg)'].head())

0    72.121128
1    83.007336
2    87.089664
3    69.853168
4    68.038800
Name: Weight (in kg), dtype: float64


* Convert Height from inches to cm

In [25]:
df['Height (in cm)'] = df['Height'].str.replace("'", '.').str.replace('"', '')
df['Height (in cm)'] = (df['Height (in cm)'].astype(float) * (30.48)).round(2)
print(df['Height (in cm)'].head())

0    173.74
1    188.98
2    188.98
3    155.75
4    179.83
Name: Height (in cm), dtype: float64


* Convert Joined column to day, month and year columns

In [51]:
#print(df.Joined.head(20))
#print(df.Joined.unique())
df['Joined'] = pd.to_datetime(df['Joined'])
#print(df.Joined.dtype)
df['Year'] = df['Joined'].dt.year
df['Month'] = df['Joined'].dt.month
df['Day'] = df['Joined'].dt.day

print(df[['Joined', 'Year', 'Month', 'Day']].head())

      Joined  Year  Month  Day
0 2004-07-01  2004      7    1
1 2018-07-10  2018      7   10
2 2014-07-16  2014      7   16
3 2015-08-30  2015      8   30
4 2017-08-03  2017      8    3


* Convert wage, value, Relase clause from string to int (M =million)

In [102]:
def convertWage(wage):
    if 'K' in wage:
        wage = wage.replace('K', '')
        wage = int(wage) * 1000
    else:
        wage = int(wage)
        
    return wage

def convertValue(value):
    value = value.lstrip('€')
    if 'M' in value:
        value = value.replace('M', '')
        value = float(value) * 1000000
    elif 'K' in value:
        value = value.replace('K', '')
        value = float(value) * 1000
        
    return value

def convertReleaseClause(rc):
    rc = rc.lstrip('€')
    if 'M' in rc:
        rc = rc.strip('M')
        rc = int(float(rc) * 1000000)
    elif 'K' in rc:
        rc = rc.strip('K')
        rc = int(float(rc) * 1000)
    
    return int(rc)

#print(df[['Wage', 'Value', 'Release Clause']].head())
#print(df.Wage.unique())

df['Wage (in euro)'] = df.Wage.str.replace('€', '')
df['Wage (in euro)'] = df['Wage (in euro)'].apply(convertWage)
print(df['Wage (in euro)'])

#df['Value'].unique()
df['Value (in euro)'] = df['Value'].apply(convertValue)
print(df['Value (in euro)'])

#df['Release Clause'].head()
#df['Release Clause'].unique()
df['Release Clause (in euro)'] = df['Release Clause'].apply(convertReleaseClause)
print(df['Release Clause (in euro)'])

0        560000
1        220000
2        125000
3        370000
4        270000
          ...  
18974      1000
18975       500
18976      1000
18977       500
18978       500
Name: Wage (in euro), Length: 18979, dtype: int64
0        67500000.0
1        46000000.0
2        75000000.0
3        87000000.0
4        90000000.0
            ...    
18974       35000.0
18975       60000.0
18976       40000.0
18977       60000.0
18978       60000.0
Name: Value (in euro), Length: 18979, dtype: object
0        138400000
1         75900000
2        159400000
3        161000000
4        166500000
           ...    
18974        57000
18975       165000
18976        70000
18977       165000
18978       167000
Name: Release Clause (in euro), Length: 18979, dtype: int64


* Player playing for current club more than 10 years

In [126]:
df.Joined = pd.to_datetime(df['Joined'])
df.Joined = df.Joined.dt.date
today = date.today()

days_in_year = 365.2425
for idx, join in df.Joined.iteritems():
    if(int((today - join).days / days_in_year) >= 10):
        df.loc[idx, '10 Years'] = True
    else:
        df.loc[idx, '10 Years'] = False
    
print(df[['Joined', '10 Years']].head())   

       Joined 10 Years
0  2004-07-01     True
1  2018-07-10    False
2  2014-07-16    False
3  2015-08-30    False
4  2017-08-03    False
